In [ ]:
from my_libs import *

In [ ]:



class datapairtrade(bt.feeds.PandasData):
    lines = ('datetime','open','high','low','close','volume','openinterest','log_ret','log_ret_mv','volatility')
    params = (
            ('datetime', None),
            ('open', -1),
            ('high', -1),
            ('low', -1),
            ('close', -1),
            ('volume', -1),
            ('openinterest', -1),
            ('log_ret', -1),
        ('log_ret_mv', -1),
        ('volatility', -1),
        
        
        )
    
class datapairtrade_table(bt.feeds.PandasData):
    lines = ('datetime','relative','relative_mv','z_score',"slope")
    params = (
        ('datetime', None),
            ('relative', -1),
        ('relative_mv', -1),
        ('z_score', -1),
        ('slope', -1)
    
        )
    
    
def slope(data_frame,table, look_back_window = 3):
    # pay attention to the use of this function in apply method
        i = table.index.get_loc(data_frame.name)

        if i < window:
            return np.NaN
        else:
            return stats.linregress(table[stock1+"_log_ret"].iloc[(i-window):i],table[stock2+"_log_ret"].iloc[(i-window):i])[0]

def backtest_pair(i,j, strategy, capital = 2000, method = "day",back_day = 80,window = 3 ):

#     try:
    robinhood = robingateway()
    
    price1 = get_price_data([i], method=method,robinhood=robinhood,back_day=back_day )
    price1 = price1.set_index("TimeStamp")

    price2 = get_price_data([j], method=method,robinhood=robinhood,back_day=back_day )
    price2 = price2.set_index("TimeStamp")
    price_table = pd.DataFrame()
    #--------------------------------------------------------------

    


    price1.loc[:,"log_ret"] = log(price1.Close / price1.Close.shift(1))
    price2.loc[:,"log_ret"] = log(price1.Close / price2.Close.shift(1))


    price1.loc[:, "log_ret_mv"] = price1.log_ret.rolling(window).mean()
    price2.loc[:, "log_ret_mv"] = price2.log_ret.rolling(window).mean()

    price1 = price1.fillna(method="bfill")
    price2 = price2.fillna(method="bfill")
    
    price_table.loc[:,"relative"] = price1.log_ret_mv / price2.log_ret_mv

    price_table.loc[:,"relative_mv"] = price_table["relative"].rolling(window, min_periods=window).mean()
    price_table["relative_mv"] = abs(price_table["relative_mv"])
   

    for i in range(len(price1)-window):
        i += window
        price_table.loc[price_table.iloc[i].name,"slope"] = stats.linregress(price1.log_ret.iloc[:i],price2.log_ret.iloc[:i])[0]





    price1.loc[:,"volatility"] = price1.log_ret.rolling(window,min_periods=window-5).std()
    price2.loc[:,"volatility"] = price2.log_ret.rolling(window,min_periods=window-5).std()


    price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
    price_table = price_table.fillna(method="bfill")




    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    cerebro.broker.setcommission(commission=0.0)



    # Create a Data Feed
    data1 = datapairtrade(dataname=price1)
    data2 = datapairtrade(dataname=price2)
    data3 = datapairtrade_table(dataname=price_table)


    cerebro.adddata(data1)
    cerebro.adddata(data2)
    cerebro.adddata(data3)
    cerebro.broker.setcash(capital)
    init = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    post = cerebro.broker.getvalue()
    if post > init:
        print ("Good Trade")

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
#     cerebro.plot()
            
#     except Exception as e:
#         print (e)
#         return

class pair_trade(bt.Strategy):
    
    
    
    def buy_line(self,data_frame):
        data_frame = data_frame.slope
        if data_frame>0.5:
            return -1.25
        elif data_frame>0.75:
            return -1.75
        elif data_frame:
            return -2.25
        elif data_frame<-0.75:
            return -2.75
        else:
            return -2
    def sell_line(self,data_frame):
        data_frame = data_frame.slope
        if data_frame>0.5:
            return 1.25
        elif data_frame>0.75:
            return 1.75
        elif data_frame:
            return 2.25
        elif data_frame<-0.75:
            return 2.75
        else:
            return 2


    def __init__(self):
        self.buy_lines = None
        self.sell_lines = None
    
        #self.adxr = bt.indicators.ADXR()
        #self.cci = bt.indicators.CCI()
        #self.macd = bt.talib.MACD(self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        #self.mfi = bt.talib.MFI(self.data.High, self.data.Low, self.data.Close, self.data.Volume,timeperiod=14)
        #self.rocp = bt.talib.ROCP(self.data.Close, timeperiod=10)
       
        

    def next(self):
        print self.broker.getcash()
        self.buy_lines = self.buy_line(self.datas[2])
        self.sell_lines = self.sell_line(self.datas[2])
        print (str(self.datas[2].datetime.date(0)) +" Z-score:" + str(self.datas[2].slope[0]) + " buy line: " + str(self.buy_lines) + " sell line: "+str(self.sell_lines))
   
        if self.datas[2].z_score < self.buy_lines:
            self.buy(size = np.ceil(((self.broker.getcash()/2)/(1+self.datas[2].relative_mv)/self.datas[0].close[0]))
                     )
            self.sell(size = np.ceil(((self.broker.getcash()/2) \
                                /(1+self.datas[2].relative_mv)*self.datas[2].relative_mv[0]/self.datas[1].close[0]))  
                )
            print str(self.data.datetime.date()) + ": buy signal" 

        if self.datas[2].z_score > self.sell_lines:
            self.sell(size = np.ceil(((self.broker.getcash()/2)/(1+self.datas[2].relative_mv)/self.datas[0].close))
                    )
            self.buy(size = np.ceil(((self.broker.getcash()/2)\
                               /(1+self.data.relative_mv)*self.datas[2].relative_mv[0]/self.datas[1].close))  
              )
            print str(self.data.datetime.date()) + ": sell signal"
        

                    
                    
                        



In [ ]:
i = "AAPL"
j="AMZN"
backtest_pair(i,j,pair_trade,back_day = 80)

In [3]:
from my_libs import *
i = "AAPL"
j="AMZN"
pair_trade(i,j,robinhood = robingateway(),initial = 2000,method = 'day',data_len=200)

Rolling window = 3, Backtest length= 200
All price data of Close is actually Adj Close
Connection Successful
Finished AAPL
All price data of Close is actually Adj Close
Connection Successful
Finished AMZN
Log return done, 10%
Signal Line done, 30%
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
2000.0
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999999975
-310.01999999

,AAPL_close,AMZN_close,AAPL_log_ret,AMZN_log_ret,AAPL_log_ret_mv,AMZN_log_ret_mv,relative,relative_mv,AAPL_volatility,AMZN_volatility,...,cash,slope,buy_line,sell_line,AAPL_shares,AMZN_shares,AAPL_value,AMZN_value,p_L,total_p_L
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2019-05-01,210.52,1911.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000.00,NaN,-2.25,2.25,0.0,0.0,NaN,NaN,NaN,164.212763
2019-05-02,209.15,1900.82,-0.006529,-0.005613,NaN,NaN,NaN,NaN,NaN,NaN,...,2000.00,NaN,-2.25,2.25,0.0,0.0,0.00,0.00,NaN,164.212763
2019-05-03,211.75,1962.46,0.012355,0.031913,NaN,NaN,NaN,NaN,0.013353,0.026535,...,2000.00,NaN,-2.25,2.25,0.0,0.0,0.00,0.00,0.000000,164.212763
2019-05-06,208.48,1950.55,-0.015563,-0.006087,-0.003246,0.006738,-0.481755,NaN,0.014246,0.021804,...,2000.00,NaN,-2.25,2.25,0.0,0.0,0.00,0.00,-0.000000,164.212763
2019-05-07,202.86,1921.00,-0.027327,-0.015266,-0.010179,0.003520,-2.891490,NaN,0.020381,0.025014,...,2000.00,1.456803,-1.25,1.25,0.0,0.0,0.00,0.00,-0.000000,164.212763
2019-05-08,202.90,1917.77,0.000197,-0.001683,-0.014231,-0.007679,1.853342,0.506634,0.013810,0.006930,...,2000.00,1.220099,-1.25,1.25,0.0,0.0,0.00,0.00,0.000000,164.212763
2019-05-09,200.72,1899.87,-0.010802,-0.009378,-0.012644,-0.008775,1.440869,0.134240,0.013854,0.006811,...,2000.00,0.481702,-2.25,2.25,0.0,0.0,0.00,0.00,-0.000000,164.212763
2019-05-10,197.18,1889.98,-0.017794,-0.005219,-0.009466,-0.005427,1.744453,1.679555,0.009070,0.003852,...,2000.00,0.482606,-2.25,2.25,0.0,0.0,0.00,0.00,-0.000000,164.212763
2019-05-13,185.72,1822.68,-0.059877,-0.036258,-0.029491,-0.016952,1.739709,1.641677,0.026546,0.016849,...,2000.00,0.241493,-2.25,2.25,0.0,0.0,0.00,0.00,-0.000000,164.212763


In [ ]:
#backup


robinhood = robingateway()
        price1 = get_price_data([i], method=method,robinhood=robinhood,back_day=back_day )
        price1 = price1.set_index("TimeStamp")
        
        price2 = get_price_data([j], method=method,robinhood=robinhood,back_day=back_day )
        price2 = price2.set_index("TimeStamp")
    
    #--------------------------------------------------------------
    
        price1 = price1.fillna(method="bfill")
        price2 = price2.fillna(method="bfill")
        

        price1.loc[:,"log_ret"] = log(price1.Close / price1.Close.shift(1))
        price2.loc[:,"log_ret"] = log(price1.Close / price2.Close.shift(1))
       

        price1.loc[:, "log_ret_mv"] = price1.log_ret.rolling(window).mean()
        price2.loc[:, "log_ret_mv"] = price2.log_ret.rolling(window).mean()
        

        price_table.loc[:,"relative"] = price1.log_ret_mv / price2.log_ret_mv

        price_table.loc[:,"relative_mv"] = price_table["relative"].rolling(window, min_periods=window).mean()
        price_table["relative_mv"] = abs(price_table["relative_mv"])
        
        price_table.loc[:,"log_ret_slope"] = stats.linregress(price1.log_ret.iloc[])


        price_table.fillna(method="bfill")

            
    
        price1.loc[:,"volatility"] = price1.log_ret.rolling(window,min_periods=window-5).std()
        price2.loc[:,"volatility"] = price2.log_ret.rolling(window,min_periods=window-5).std()


        price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
        price_table.fillna(method="bfill")

In [ ]:
        
def backtest_pair(stock1,stock2, strategy, capital = 2000, method = "day",back_day = 80,window = 3 ):
    
    class datapairtrade(bt.feeds.PandasData):
        lines = ('datetime',"close1","close2","log_ret1","log_ret2","log_ret_mv1","log_ret_mv2",'relative','relative_mv',"volatility1","volatility2","z_score","slope","high1","high2","low1","low2","open1","open2")
        params = (
                ('datetime', None),
                ("close1", -1),
                ("close2", -1),
                ("log_ret1", -1),
                ("log_ret2", -1),
                ("log_ret_mv1", -1),
            ("log_ret_mv2", -1),
                ('relative', -1),
                ('relative_mv', -1),
            ("volatility1", -1),
            ("volatility2", -1),
            ('z_score', -1),
            ('slope', -1),
            ("open1", -1),
                ("open2", -1),
            ("high1", -1),
                ("high2", -1),
            ("low1", -1),
                ("low2", -1)
            


            ) 

In [ ]:
## backup

price1 = get_price_data([stock1],method="day",start_date=datetime.now()-timedelta(days=back_day),end_date=datetime.now())
    price2 = get_price_data([stock2],method="day",start_date=datetime.now()-timedelta(days=back_day),end_date=datetime.now())
    price1 = price1.rename(columns={"Close":"close1"})
    price2 = price2.rename(columns={"Close":"close2"})
    price1 = price1.rename(columns={"Open":"open1"})
    price2 = price2.rename(columns={"Open":"open2"})
    price1 = price1.rename(columns={"High":"high1"})
    price2 = price2.rename(columns={"High":"high2"})
    price1 = price1.rename(columns={"Low":"low1"})
    price2 = price2.rename(columns={"Low":"low1"})

    price1 = price1.set_index("TimeStamp")
    price2 = price2.set_index("TimeStamp")

    price_table = pd.concat([price1["close1"], price2["close2"]], axis=1)
    price_table = price_table.fillna(price_table.shift(1))
    price_table["close1"] = price_table["close1"].astype(float)
    price_table["close2"] = price_table["close2"].astype(float)
    price_table["log_ret1"] = log(
        price_table["close1"] / price_table["close1"].shift(1))
    price_table["log_ret2"] = log(
        price_table["close2"] / price_table["close2"].shift(1))

    price_table["log_ret_mv1"] = price_table["log_ret1"].rolling(window).mean()
    price_table["log_ret_mv2"] = price_table["log_ret2"].rolling(window).mean()
#TODO
    price_table["relative"] = price_table["log_ret_mv1"] / price_table["log_ret_mv2"]
    #price_table["relative"] = abs(price_table[stock1 + "_log_ret_mv"] / price_table[stock2 + "_log_ret_mv"])

    price_table["relative_mv"] = price_table["relative"].rolling(window, min_periods=window).mean()
    price_table["relative_mv"] = abs(price_table["relative_mv"])

    price_table.relative.fillna(method="bfill")

#TODO
    price_table.relative.loc[price_table.relative == -np.inf] = \
        price_table.loc[price_table.relative != -np.inf].sort_values("relative", ascending=True).iloc[0][
            "relative"]
    # price_table.relative.loc[price_table.relative==np.NaN]=price_table.relative.shift(1)
    price_table.relative.loc[price_table.relative == np.inf] = \
        price_table.loc[price_table.relative != np.inf].sort_values("relative", ascending=False).iloc[0][
            "relative"]

    price_table["volatility1"] = price_table["log_ret1"].rolling(window,
                                                                                   min_periods=window - 5).std()
    price_table["volatility2"] = price_table["log_ret2"].rolling(window,
                                                                                   min_periods=window - 5).std()
    # price_table=price_table.fillna(1)

    # price_table["relative_mv"]=price_table[stock1+"_log_ret_mv"]/price_table[stock2+"_log_ret_mv"]
    price_table["z_score"] = (price_table["relative"] - price_table["relative_mv"]) / price_table.relative.std()


    # price_table.fillna(method="bfill")
    # price_table["trade_signal"]=np.NaN

    # for i in range(window-1,len(price_table)):
    #     price_table["slope"].iloc[i] = stats.linregress(price_table[stock1+"_log_ret"]\
    #                                                     .iloc[i-(window-1):i],price_table[stock2+"_log_ret"].iloc[i-(window-1):i])[0]
    # price_table["buy_line"] = np.NaN
    # price_table["sell_line"] = np.NaN

    def slope(data_frame, table):
        i = table.index.get_loc(data_frame.name)

        if i < window:
            return np.NaN
        else:
            return stats.linregress(table["log_ret1"].iloc[(i - window):i],
                                    table["log_ret2"].iloc[(i - window):i])[0]

    price_table["slope"] = price_table.apply(slope, axis=1, args=(price_table,))


    price_table= price_table.fillna(method="bfill")
    
###################################################################
    
    
      def next(self):
        self.buy_lines = self.buy_line(self.data)
        self.sell_lines = self.sell_line(self.data)
        print (str(self.data.datetime.date(0)) +" Z-score:" + str(self.data.slope[0]) + " buy line: " + str(self.buy_lines) + " sell line: "+str(self.sell_lines))
   
        if self.data.z_score < self.buy_lines:
            self.buy(size = np.floor(((self.broker.getcash()/2)/(1+self.data.relative_mv)/self.data.close1[0]))
                     ,price = self.data.close1[0])
            self.sell(size = np.floor(((self.broker.getcash()/2) \
                                /(1+self.data.relative_mv)*self.data.relative_mv[0]/self.data.close2[0]))  
                ,price = self.data.close2[0])
            print str(self.data.datetime.date()) + ": buy signal"

        if self.data.z_score > self.sell_lines:
            self.sell(size = np.floor(((self.broker.getcash()/2)/(1+self.data.relative_mv)/self.data.close1[0]))
                     ,price = self.data.close1[0])
            self.buy(size = np.floor(((self.broker.getcash()/2)\
                               /(1+self.data.relative_mv)*self.data.relative_mv[0]/self.data.close2[0]))  
                ,price = self.data.close2[0])
            print str(self.data.datetime.date()) + ": sell signal"
        print self.broker.getcash()